In [1]:
versioninfo()

Julia Version 1.5.1
Commit 697e782ab8 (2020-08-25 20:08 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


In [ ]:
]activate .

In [2]:
]instantiate

In [3]:
using Squareness

## Simple Newton Method

In [4]:
function _issquare_with_simple_newton(x::I) where {I <: Integer}
    x == 0 && return true
    xtz = trailing_zeros(x)
    isodd(xtz) && return false
    x >>= unsigned(xtz)
    o_xrt::I = x
    xrt::I = (x + 0x1) >> 0x1
    xrt == 0 && return false  # overflow
    while xrt < o_xrt
        o_xrt = xrt
        xrt = (xrt + x ÷ xrt) >> 0x1
    end
    xrt * xrt == x
end

issquare_with_simple_newton(x::Signed) = x < 0 ? false : _issquare_with_simple_newton(x % Unsigned)
issquare_with_simple_newton(x::Unsigned) = _issquare_with_simple_newton(x)
issquare_with_simple_newton(x::BigInt) = x < 0 ? false : _issquare_with_simple_newton(x)

issquare_with_simple_newton (generic function with 3 methods)

### Simple Check

In [5]:
all(issquare(x) == issquare_with_simple_newton(x) == Squareness.issquare_with_isqrt(x) for x in rand(Int, 10000))

true

In [6]:
all(issquare(x) == issquare_with_simple_newton(x) == Squareness.issquare_with_isqrt(x) for x in rand(Int32, 10000))

true

## Benchmark

In [7]:
using BenchmarkTools

### Compare `issquare()`(with P-adic Newton) vs `issquare_with_isqrt()` vs `issquare_with_simple_newton()`

#### `Int64`

In [8]:
rand_int64() = rand(Int64) & typemax(Int64)

rand_int64 (generic function with 1 method)

In [9]:
@benchmark issquare(n) setup=(n=rand_int64())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.707 ns (0.00% GC)
  median time:      5.233 ns (0.00% GC)
  mean time:        4.847 ns (0.00% GC)
  maximum time:     38.426 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [10]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int64())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.654 ns (0.00% GC)
  median time:      7.673 ns (0.00% GC)
  mean time:        7.752 ns (0.00% GC)
  maximum time:     40.545 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [11]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int64())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.645 ns (0.00% GC)
  median time:      326.547 ns (0.00% GC)
  mean time:        231.232 ns (0.00% GC)
  maximum time:     591.443 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

#### `Int64` × 100

In [12]:
rand_int64(N) = rand(Int64, N) .& typemax(Int64)

rand_int64 (generic function with 2 methods)

In [13]:
@benchmark issquare.(n) setup=(n=rand_int64(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     483.741 ns (0.00% GC)
  median time:      593.013 ns (0.00% GC)
  mean time:        616.227 ns (1.23% GC)
  maximum time:     16.795 μs (95.84% GC)
  --------------
  samples:          10000
  evals/sample:     193

In [14]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int64(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     359.348 ns (0.00% GC)
  median time:      374.696 ns (0.00% GC)
  mean time:        396.102 ns (2.22% GC)
  maximum time:     17.237 μs (97.72% GC)
  --------------
  samples:          10000
  evals/sample:     207

In [15]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int64(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     18.370 μs (0.00% GC)
  median time:      25.505 μs (0.00% GC)
  mean time:        25.590 μs (0.00% GC)
  maximum time:     50.502 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

#### `Int32`

In [16]:
rand_int32() = rand(Int32) & typemax(Int32)

rand_int32 (generic function with 1 method)

In [17]:
@benchmark issquare(n) setup=(n=rand_int32())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.715 ns (0.00% GC)
  median time:      4.312 ns (0.00% GC)
  mean time:        4.195 ns (0.00% GC)
  maximum time:     35.114 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [18]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int32())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.934 ns (0.00% GC)
  median time:      8.258 ns (0.00% GC)
  mean time:        8.374 ns (0.00% GC)
  maximum time:     32.832 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [19]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int32())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.469 ns (0.00% GC)
  median time:      122.975 ns (0.00% GC)
  mean time:        86.933 ns (0.00% GC)
  maximum time:     198.754 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

#### `Int32` × 100

In [20]:
rand_int32(N) = rand(Int32, N) .& typemax(Int32)

rand_int32 (generic function with 2 methods)

In [21]:
@benchmark issquare.(n) setup=(n=rand_int32(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     449.474 ns (0.00% GC)
  median time:      532.982 ns (0.00% GC)
  mean time:        561.447 ns (1.41% GC)
  maximum time:     17.276 μs (95.99% GC)
  --------------
  samples:          10000
  evals/sample:     196

In [22]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int32(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     401.626 ns (0.00% GC)
  median time:      429.286 ns (0.00% GC)
  mean time:        449.911 ns (1.99% GC)
  maximum time:     23.725 μs (97.50% GC)
  --------------
  samples:          10000
  evals/sample:     203

In [23]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int32(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     5.872 μs (0.00% GC)
  median time:      8.090 μs (0.00% GC)
  mean time:        8.263 μs (0.00% GC)
  maximum time:     25.306 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     6

#### `Int16`

In [24]:
rand_int16() = rand(Int16) & typemax(Int16)

rand_int16 (generic function with 1 method)

In [25]:
@benchmark issquare(n) setup=(n=rand_int16())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.710 ns (0.00% GC)
  median time:      9.817 ns (0.00% GC)
  mean time:        7.643 ns (0.00% GC)
  maximum time:     44.694 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [26]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int16())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     6.377 ns (0.00% GC)
  median time:      7.644 ns (0.00% GC)
  mean time:        7.884 ns (0.00% GC)
  maximum time:     47.369 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [27]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int16())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.710 ns (0.00% GC)
  median time:      54.104 ns (0.00% GC)
  mean time:        39.707 ns (0.00% GC)
  maximum time:     180.570 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

#### `Int16` × 100

In [28]:
rand_int16(N) = rand(Int16, N) .& typemax(Int16)

rand_int16 (generic function with 2 methods)

In [29]:
@benchmark issquare.(n) setup=(n=rand_int16(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     388.495 ns (0.00% GC)
  median time:      468.947 ns (0.00% GC)
  mean time:        488.022 ns (1.68% GC)
  maximum time:     18.304 μs (97.25% GC)
  --------------
  samples:          10000
  evals/sample:     200

In [30]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int16(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     399.634 ns (0.00% GC)
  median time:      432.106 ns (0.00% GC)
  mean time:        449.524 ns (1.82% GC)
  maximum time:     17.340 μs (97.41% GC)
  --------------
  samples:          10000
  evals/sample:     202

In [31]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int16(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     2.554 μs (0.00% GC)
  median time:      3.472 μs (0.00% GC)
  mean time:        3.575 μs (0.00% GC)
  maximum time:     12.040 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     9

#### `Int128`

In [32]:
rand_int128() = rand(Int128) & typemax(Int128)

rand_int128 (generic function with 1 method)

In [33]:
@benchmark issquare(n) setup=(n=rand_int128())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.953 ns (0.00% GC)
  median time:      25.470 ns (0.00% GC)
  mean time:        18.249 ns (0.00% GC)
  maximum time:     83.659 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000

In [34]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_int128())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     12.737 ns (0.00% GC)
  median time:      13.652 ns (0.00% GC)
  mean time:        13.770 ns (0.00% GC)
  maximum time:     51.483 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     998

In [35]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_int128())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.883 ns (0.00% GC)
  median time:      1.844 μs (0.00% GC)
  mean time:        1.240 μs (0.00% GC)
  maximum time:     2.569 μs (0.00% GC)
  --------------
  samples:          4032
  evals/sample:     1000

#### `Int128` × 100

In [36]:
rand_int128(N) = rand(Int128, N) .& typemax(Int128)

rand_int128 (generic function with 2 methods)

In [37]:
@benchmark issquare.(n) setup=(n=rand_int128(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     1.547 μs (0.00% GC)
  median time:      2.007 μs (0.00% GC)
  mean time:        2.039 μs (0.83% GC)
  maximum time:     170.647 μs (98.65% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [38]:
@benchmark Squareness.issquare_with_isqrt.(n) setup=(n=rand_int128(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     1.441 μs (0.00% GC)
  median time:      1.546 μs (0.00% GC)
  mean time:        1.614 μs (1.28% GC)
  maximum time:     208.554 μs (98.75% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [39]:
@benchmark issquare_with_simple_newton.(n) setup=(n=rand_int128(100))

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  2
  --------------
  minimum time:     95.538 μs (0.00% GC)
  median time:      129.065 μs (0.00% GC)
  mean time:        130.734 μs (0.00% GC)
  maximum time:     301.311 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

#### `BigInt`

In [40]:
rand_mpz() = (big(rand(UInt128)) << rand(0:128)) + rand(UInt128)

rand_mpz (generic function with 1 method)

In [41]:
# @benchmark issquare(n) setup=(n=rand_mpz())
@benchmark Squareness.issquare_with_padic_newton(n) setup=(n=rand_mpz())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.611 ns (0.00% GC)
  median time:      304.205 μs (1.48% GC)
  mean time:        315.324 μs (3.17% GC)
  maximum time:     619.947 μs (1.41% GC)
  --------------
  samples:          16
  evals/sample:     999

In [42]:
@benchmark Squareness.issquare_with_isqrt(n) setup=(n=rand_mpz())

BenchmarkTools.Trial: 
  memory estimate:  88 bytes
  allocs estimate:  5
  --------------
  minimum time:     136.506 ns (0.00% GC)
  median time:      253.256 ns (0.00% GC)
  mean time:        394.322 ns (19.49% GC)
  maximum time:     119.505 μs (75.92% GC)
  --------------
  samples:          10000
  evals/sample:     858

In [43]:
@benchmark issquare_with_simple_newton(n) setup=(n=rand_mpz())

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     9.077 ns (0.00% GC)
  median time:      26.642 μs (0.00% GC)
  mean time:        35.133 μs (23.66% GC)
  maximum time:     127.642 μs (60.57% GC)
  --------------
  samples:          143
  evals/sample:     999